In [1]:
# General imports
import os
from os.path import dirname, abspath
import yaml
from copy import deepcopy
from omegaconf import DictConfig
from functools import partial
from datetime import date
import timeit
from tests.optuna.optuna_base_config import update_configuration

# Current project imports
import goli
from goli.config._loader import load_datamodule, load_metrics, load_architecture, load_predictor, load_trainer

# Optuna
import optuna
from optuna.trial import TrialState

# WandB
import wandb

# Set up the working directory
MAIN_DIR = dirname(dirname(abspath(goli.__file__)))
# CONFIG_FILE = "tests/mtl/config_micro_ZINC_mtl_test_3_tasks_pyg.yaml"
#CONFIG_FILE = "tests/mtl/config_ipu_9atoms.yaml"
CONFIG_FILE = "tests/mtl/config_ipu_allsizes.yaml"
# CONFIG_FILE = "tests/mtl/config_ipu_reproduce.yaml"
OPTUNA_CONFIG_FILE = "tests/optuna/optuna_base_config.py"
os.chdir(MAIN_DIR)

def main(cfg: DictConfig, trial, run_name="main") -> None:
    st = timeit.default_timer()

    cfg = deepcopy(cfg)

    # Load and initialize the dataset
    datamodule = load_datamodule(cfg)

    # Initialize the network
    model_class, model_kwargs = load_architecture(
        cfg,
        in_dim_nodes=datamodule.num_node_feats_with_positional_encoding,
        in_dim_edges=datamodule.num_edge_feats,
    )


    metrics = load_metrics(cfg)
    print(metrics)

    predictor = load_predictor(cfg, model_class, model_kwargs, metrics)

    print(predictor.model)
    print(predictor.summarize(max_depth=4))

    trainer = load_trainer(cfg, run_name)

    datamodule.prepare_data()
    trainer.fit(model=predictor, datamodule=datamodule)
    # Run the model training
    print("\n------------ TRAINING STARTED ------------")
    try:
        print("\n------------ TRAINING COMPLETED ------------\n\n")

    except Exception as e:
        if not cfg["constants"]["raise_train_error"]:
            print("\n------------ TRAINING ERROR: ------------\n\n", e)
        else:
            raise e

    print("\n------------ TESTING STARTED ------------")
    try:
        ckpt_path = trainer.checkpoint_callbacks[0].best_model_path
        #ckpt_path = "models_checkpoints/micro_ZINC_mtl/model-v3.ckpt"
        trainer.test(model=predictor, datamodule=datamodule) #, ckpt_path=ckpt_path)
        print("\n------------ TESTING COMPLETED ------------\n\n")

    except Exception as e:
        if not cfg["constants"]["raise_train_error"]:
            print("\n------------ TESTING ERROR: ------------\n\n", e)
        else:
            raise e

    print ("--------------------------------------------")
    print("total computation used", timeit.default_timer() - st)
    print ("--------------------------------------------")

    return trainer.callback_metrics["cv/mae/test"].cpu().item()

if __name__ == "__main__":
    #nan_checker("goli/data/QM9/micro_qm9.csv") #can be deleted
    with open(os.path.join(MAIN_DIR, CONFIG_FILE), "r") as f:
        cfg = yaml.safe_load(f)

    def objective(trial, cfg):
        cfg = deepcopy(cfg)
    
        with open(os.path.join(MAIN_DIR, OPTUNA_CONFIG_FILE), "r") as optuna_config:
            cfg = update_configuration(trial, cfg)

        run_name = 'no_name_' if not "name" in cfg["constants"] else cfg["constants"]["name"] + "_"
        run_name = run_name + date.today().strftime("%d/%m/%Y") + "_"
        for key, value in trial.params.items():
            run_name = run_name + str(key) + "=" + str(value) + "_"

        accu = main(cfg, trial, run_name=run_name[:len(run_name) - 1])
        wandb.log(data={"cv/mae/test": accu})
        wandb.finish()
        return accu

    study = optuna.create_study()
    study.optimize(partial(objective, cfg=cfg), n_trials=5, timeout=600)

    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))


[I 2022-09-07 20:13:19,251] A new study created in memory with name: no-name-e5988481-195a-4f9a-b3ed-223dfa1f561f
/home/oleksandr/.venv/goli_ipu/lib/python3.8/site-packages/optuna/distributions.py:560: UserWarning: The distribution is specified by [16, 124] and step=16, but the range is not divisible by `step`. It will be replaced by [16, 112].
  warnings.warn(
/tmp/ipykernel_1736612/3964897578.py:106: LightningDeprecationWarning: The `LightningModule.summarize` method is deprecated in v1.5 and will be removed in v1.7. Use `pytorch_lightning.utilities.model_summary.summarize` instead.
  accu = main(cfg, trial, run_name=run_name[:len(run_name) - 1])


{'homo': {'mae': mean_absolute_error, 'pearsonr': pearson_corrcoef}, 'alpha': {'mae': mean_absolute_error, 'pearsonr': pearson_corrcoef}, 'cv': {'mae': mean_absolute_error, 'pearsonr': pearson_corrcoef}}
MTL_DGL_GNN
-------------
    pre-NN(depth=1, ResidualConnectionNone)
        [FCLayer[55 -> 32]
    
    pre-NN-edges(depth=1, ResidualConnectionNone)
        [FCLayer[13 -> 16]
    
    GNN(depth=4, ResidualConnectionSimple(skip_steps=1))
        GPSLayerPyg[32 -> 16 -> 16 -> 16 -> 32]
        -> Pooling(['sum', 'mean', 'max']) -> FCLayer(96 -> 32, activation=None)
    

   | Name                              | Type                      | Params
---------------------------------------------------------------------------------
0  | model                             | FullGraphMultiTaskNetwork | 30.0 K
1  | model.pre_nn                      | FeedForwardNN             | 1.8 K 
2  | model.pre_nn.activation           | ReLU                      | 0     
3  | model.pre_nn.residual_layer  

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: alexdymov (multitask-gnn). Use `wandb login --relogin` to force relogin


/home/oleksandr/.venv/goli_ipu/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:572: LightningDeprecationWarning: Trainer argument `terminate_on_nan` was deprecated in v1.5 and will be removed in 1.7. Please use `Trainer(detect_anomaly=True)` instead.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: True, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/oleksandr/.venv/goli_ipu/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1836: UserWarning: IPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='ipu', devices=4)`.
  rank_zero_warn(
2022-09-07 20:13:21.426 | INFO     | goli.data.datamodule:prepare_data:669 - Reading data for task 'homo'
2022-09-07 20:13:21.438 | INFO     | goli.data.datamodule:prepare_data:669 - Reading data for task 'alpha'
2022-09-07 20:13:21.448 | INFO     | goli.data.datamodule:prepare_data:669 - Reading data for task 'cv'
2022

mols to ids:   0%|          | 0/3015 [00:00<?, ?it/s]

...

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
  alpha/MSELossIPU/test     1.7398873567581177
     alpha/loss/test        1.7398873567581177
     alpha/mae/test         0.9153303503990173
  alpha/mean_pred/test     -0.039938200265169144
 alpha/mean_target/test    -0.17942453920841217
 alpha/median_pred/test    -0.04046527296304703
alpha/median_target/test     -0.05877685546875
   alpha/pearsonr/test     0.010962050408124924
   alpha/std_pred/test     0.005124276038259268
  alpha/std_target/test     1.3153568506240845
   cv/MSELossIPU/test       1.4108092784881592
      cv/loss/test          1.4108092784881592
       cv/mae/test          0.8540242314338684
    cv/mean_pred/test      -0.06344887614250183
   cv/mean_target/test     -0.11077180504798889
 

alpha/MSELossIPU/test,▁
alpha/MSELossIPU/train,▃▃▁▁▄▄▄▄██▅▅▂▂██▇▇▂▂▃▃▁▁▄▄▄▄██▅▅▂▂██▇▇▂▂
alpha/MSELossIPU/val,█▁
alpha/loss/test,▁
alpha/loss/val,█▁
alpha/mae/test,▁
alpha/mae/train,▃▃▁▁▅▅▄▄██▄▄▂▂▄▄▃▃▃▃▃▃▁▁▅▅▄▄██▄▄▂▂▄▄▃▃▃▃
alpha/mae/val,█▁
alpha/mean_pred/test,▁
alpha/mean_pred/train,▇▇▇▇██████▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▆▆▆▅▅▄▄▃▃▁▁
alpha/mean_pred/val,█▁


[I 2022-09-07 20:14:28,980] Trial 4 finished with value: 0.8540242314338684 and parameters: {'gnn.hid': 32, 'gnn.depth': 3}. Best is trial 0 with value: 0.8514640927314758.


Study statistics: 
  Number of finished trials:  5
  Number of complete trials:  5
Best trial:
  Value:  0.8514640927314758
  Params: 
    gnn.hid: 16
    gnn.depth: 4


In [8]:
# Optimization history plot
optuna.visualization.plot_optimization_history(study)

In [9]:
# Slice plot
optuna.visualization.plot_slice(study)

In [10]:
# Contour plot
optuna.visualization.plot_contour(study, params=['gnn.hid', 'gnn.depth'])